### Movie Review Dataset
We'll start by loading in the IMDB movie review dataset from keras. This dataset contains 25,000 reviews from IMDB where each one is already preprocessed and  has a label as either positive or negative. Each review is encoded by integers that represents how common a word is in the entire dataset. For example. a word encoded by the integer 3 means that it is the 3rd most common word in the dataset

In [22]:
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words= VOCAB_SIZE)

In [23]:
# Let's look at one review
train_data[0]

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 22665,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 21631,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 19193,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 10311,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 31050,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 12118,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5

In [24]:
print(len(train_data[1]))
print(len(train_data[9]))

189
130


### More Preprocessing
If we have a look at some of our loaded in reviews, we'll notice that they are different lengths. This is an issue, we cannot pass different length data into our neural network.Therefore, we must make each review the same length. To do this, we will follow the procedure below:
* If the review is greater than 250 words then trim off the extra words
* If the review is less than 250 words, add the necessary 0's to make it 250 words

Luckily for us, keras has a function that can do this for us:



In [25]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

In [26]:
print(len(train_data[1]))
print(len(train_data[9]))

250
250


### Creating the Model
Now, its time to create the model. We'll use a word embedding layer as the first layer in our model and add a LSTM layer afterwards that feeds into a dense node to get our predicted sentiment.
32 stands for the output dimension of the vectors generated by the embedding layer. We can change this value if we'd like.

In [27]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
# The reason we have used dense with the sigmoid activation function is:
# We are trying to predict the sentiment of this.i.e if we have the sentiment btw 0 and 1
# If for a data it is less than 0.5, we can classify as negative
# Sigmoid activation function squishes our value btw 0 and 1

In [28]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 32)          2834688   
                                                                 
 lstm_1 (LSTM)               (None, 32)                8320      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2843041 (10.85 MB)
Trainable params: 2843041 (10.85 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Training
Now, its time to compile and train the model

In [29]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 34s 51ms/step - loss: 0.4472 - acc: 0.7822 - val_loss: 0.3064 - val_acc: 0.8798
Epoch 2/10
625/625 [==============================] - 14s 23ms/step - loss: 0.2588 - acc: 0.9004 - val_loss: 0.3321 - val_acc: 0.8584
Epoch 3/10
625/625 [==============================] - 10s 16ms/step - loss: 0.1987 - acc: 0.9273 - val_loss: 0.2927 - val_acc: 0.8846
Epoch 4/10
625/625 [==============================] - 8s 14ms/step - loss: 0.1632 - acc: 0.9423 - val_loss: 0.3749 - val_acc: 0.8624
Epoch 5/10
625/625 [==============================] - 9s 14ms/step - loss: 0.1330 - acc: 0.9547 - val_loss: 0.3412 - val_acc: 0.8818
Epoch 6/10
625/625 [==============================] - 8s 12ms/step - loss: 0.1163 - acc: 0.9613 - val_loss: 0.3275 - val_acc: 0.8810
Epoch 7/10
625/625 [==============================] - 9s 15ms/step - loss: 0.0964 - acc: 0.9681 - val_loss: 0.3707 - val_acc: 0.8834
Epoch 8/10
625/625 [==============================] - 9s 15ms/step

And we'll evaluate the model on our training data to see how well it performs

In [30]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 4s 5ms/step - loss: 0.4878 - acc: 0.8610
[0.4878470301628113, 0.8610000014305115]


So, we're scoring somewhere in the mid-high 80's. Not bad for a simple recurrent network.

### Making Predictions
Now, let's use our network to make predictions on our own reviews.

Since our reviews are encoded, we'll need to convert any review that we write into that form so the network can understand it. To do that, we'll load the encodings from the dataset adn use them to encode our own data.

In [31]:
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = tf.keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]

text = 'that movie was just amazing, so amazing'
encoded = encode_text(text)
print(encoded)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0  12  17  13  4

In [32]:
# WHile we are at it, lets make a decode function

In [33]:
reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
  PAD = 0
  text = ""
  for num in integers:
    if num != PAD:
      text += reverse_word_index[num] + " "

  return text[:-1]

print(decode_integers(encoded))

that movie was just amazing so amazing


In [37]:
# Now, time to make a prediction

def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1, 250))
  pred[0] = encoded_text
  result = model.predict(pred)
  print(result[0])

positive_review = "that movie was so awesome! I really loved it and would watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)

1/1 [==============================] - 0s 21ms/step
[0.8952339]
1/1 [==============================] - 0s 19ms/step
[0.1833686]


It then gives us a 90 percent accuracy of positivity for the psotivie review and about 20 percent positivity for the negative review. We can also add an if statement that lets it return positive if the accuracy is over 0.5 and negative otherwise